In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.functions import col
from time import time
import sys

In [0]:
sys.path.append("/Workspace/Users/faraaz_khan@epam.com/PEI/utils")

In [0]:
from utils.common_util import normalize_columns

In [0]:
start_time = time()

In [0]:
source_path = "/Volumes/fz_catalog/landing/customers/Customer.xlsx"

In [0]:
try:
    df = (spark.read
           .option("headerRows", 1)
           .format("excel")
           .option("mode", "PERMISSIVE")
           .option("treatEmptyValuesAsNulls", "true")
           .load(source_path))
    
    if df.count() == 0:
        raise ValueError(f"Excel files are readable but contain no data rows: {source_path}")

except Exception as e:
    raise RuntimeError(f"Unexpected error while reading Excel from {source_path}: {str(e)}")

In [0]:
df_renamed = normalize_columns(df)

In [0]:
df_with_audit = df_renamed.withColumn(
    "input_file_name", col("_metadata.file_name")
).withColumn("file_modification_time", col("_metadata.file_modification_time"))

In [0]:
df_with_audit.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("fz_catalog.bronze.customers")

In [0]:
end_time = time()

In [0]:
dbutils.notebook.exit(f"Run Time: {end_time - start_time} seconds")